In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [154]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd

# Function to extract job information from a page
def extract_job_information(soup):
    job_data = []  

    job_listings = soup.find_all('li', class_='JobsList_jobListItem__JBBUV')

    for job in job_listings:
        title_elem = job.find('a', class_='JobCard_seoLink__WdqHZ')
        company_elem = job.find('div', class_='EmployerProfile_employerInfo__GaPbq EmployerProfile_employerWithLogo__R_rOX')
        location_elem = job.find('div', class_='JobCard_location__N_iYE')

        # Check if elements are found before accessing their text attribute
        title = title_elem.text.strip() if title_elem else "N/A"
        company = company_elem.text.strip() if company_elem else "N/A"
        location = location_elem.text.strip() if location_elem else "N/A"

        job_data.append({'Title': title, 'Company': company, 'Location': location})

    return job_data


driver = webdriver.Chrome(r'C:\Users\vasyl\Downloads\Chromedriver\chromedriver-win64\chromedriver.exe')

# URL of the Glassdoor login page
login_url = "https://www.glassdoor.com/profile/login_input.htm"

# Your Glassdoor credentials
username = ""
password = ""

# Open the login page
driver.get(login_url)

# Wait for the login form to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "inlineUserEmail")))

# Find the username and password input fields and submit button
username_field = driver.find_element(By.ID, "inlineUserEmail")
username_field.send_keys(username)
submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@class='Button Button']")))
submit_button.click()
time.sleep(5)

password_field = driver.find_element(By.ID, "inlineUserPassword")
password_field.send_keys(password)
submit_button2 = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@class='gd-ui-button mt-std w-100pct css-jbcabp e5tvpqr2']")))
submit_button2.click()

time.sleep(5)

# URL of the page you want to scrape
url = "https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28.htm"

# Open the webpage with Selenium
driver.get(url)

# Close modal if it exists
try:
    close_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='SVGInline modal_closeIcon']")))
    close_button.click()
except TimeoutException:
    pass  # If the modal doesn't appear, continue with the scraping

job_data_all = []  # List to store all job information

while True:
    try:
        # Wait for the modal overlay to disappear
        wait.until_not(EC.presence_of_element_located((By.CLASS_NAME, "ModalOverlay")))

        # Get the page source after waiting
        page_source = driver.page_source

        # Use BeautifulSoup to parse the HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract job information from the current page
        job_data = extract_job_information(soup)
        job_data_all.extend(job_data)
        time.sleep(5)

        # Look for the "Next" button and click it if available
        
        
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='JobsList_buttonWrapper__haBp5']")))
        next_button.click()
        time.sleep(3)
        if 'disabled' in next_button.get_attribute('class'):
            # Break the loop if the "Next" button is disabled
            break
        else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            # Wait for a short time (adjust as needed)
            time.sleep(5)

    except TimeoutException:
        # Break the loop if the modal overlay does not disappear within the specified timeout
        break

# Print all job information

df=pd.DataFrame(job_data_all)
# Close the browser
driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.124)


In [152]:
df=pd.DataFrame(job_data_all)

In [153]:
df

,Title,Company,Location
0,Machine Learning Data Scientist TS/SCI with Po...,Millennial Software5.0 ★,"McLean, VA"
1,Data Scientist,A3 Missions LLC`5.0 ★,United States
2,Data Scientist,Quiet Professionals LLC4.5 ★,"Doral, FL"
3,Data Scientist,N/A,"Suitland, MD"
4,Statistician,Princeton Consumer Research2.3 ★,"Tampa, FL"
...,...,...,...
13074,"Lead Data Scientist, Fraud Risk",Plooto4.1 ★,United States
13075,Data Science Manager,TD Bank3.9 ★,"Cherry Hill, NY"
13076,"Machine Learning Engineer, Ranking","Grammarly, Inc.4.4 ★",United States
13077,Data Scientist,Globalstar Inc.3.7 ★,"Covington, LA"


In [117]:
df=df.drop_duplicates()

In [118]:
import os 
os.makedirs('C:/Users/vasyl/Downloads', exist_ok=True)
df.to_csv("C:/Users/vasyl/Downloads/jobs2.csv")

In [121]:
df

,Title,Company,Location
0,Machine Learning Data Scientist TS/SCI with Po...,Millennial Software5.0 ★,"McLean, VA"
1,Data Scientist [Post-Doc Research Fellow],Edison Welding Institute Inc3.7 ★,"Buffalo, NY"
2,Data Scientist,A3 Missions LLC`5.0 ★,United States
3,Computational Physicist/Data Scientist,Physical Sciences Inc.3.8 ★,"Boston, MA"
4,Data Scientist,Quiet Professionals LLC4.5 ★,"Doral, FL"
...,...,...,...
10553,"QA Automation Engineer III (Cypress, AI/ML)",Ayahealthcare4.3 ★,Oregon
10554,Data Scientist,N/A,"Atlanta, GA"
12204,Data Scientist,IBM3.9 ★,"San Jose, CA"
12205,"Senior Data Scientist, Analytics & Insights",The Bill & Melinda Gates Foundation3.9 ★,"Seattle, WA"


In [123]:
def clean_text(text):
    import re
    # Remove symbols and numbers using regular expression
    cleaned_text = re.sub('[^A-Za-z\s]+', '', str(text))
    return cleaned_text
df_subset=df.copy()
df_subset['Company']=df_subset['Company'].apply(clean_text)

In [124]:
df_subset

,Title,Company,Location
0,Machine Learning Data Scientist TS/SCI with Po...,Millennial Software,"McLean, VA"
1,Data Scientist [Post-Doc Research Fellow],Edison Welding Institute Inc,"Buffalo, NY"
2,Data Scientist,A Missions LLC,United States
3,Computational Physicist/Data Scientist,Physical Sciences Inc,"Boston, MA"
4,Data Scientist,Quiet Professionals LLC,"Doral, FL"
...,...,...,...
10553,"QA Automation Engineer III (Cypress, AI/ML)",Ayahealthcare,Oregon
10554,Data Scientist,NA,"Atlanta, GA"
12204,Data Scientist,IBM,"San Jose, CA"
12205,"Senior Data Scientist, Analytics & Insights",The Bill Melinda Gates Foundation,"Seattle, WA"


In [139]:
import numpy as np
df_subset.replace("NA", np.nan, inplace=True)
df_subset.dropna(inplace=True)

In [140]:
df_subset

,Title,Company,Location
0,Machine Learning Data Scientist TS/SCI with Po...,Millennial Software,"McLean, VA"
1,Data Scientist [Post-Doc Research Fellow],Edison Welding Institute Inc,"Buffalo, NY"
2,Data Scientist,A Missions LLC,United States
3,Computational Physicist/Data Scientist,Physical Sciences Inc,"Boston, MA"
4,Data Scientist,Quiet Professionals LLC,"Doral, FL"
...,...,...,...
9022,Climate Data Scientist,ClimateAI,United States
10553,"QA Automation Engineer III (Cypress, AI/ML)",Ayahealthcare,Oregon
12204,Data Scientist,IBM,"San Jose, CA"
12205,"Senior Data Scientist, Analytics & Insights",The Bill Melinda Gates Foundation,"Seattle, WA"


In [148]:
df_subset = df_subset.reset_index(drop=True)
df_subset[['City', 'State']]=df_subset['Location'].str.split(',', 1, expand=True)
df_subset = df_subset.drop(columns=['ColumnN'])

In [149]:
import os 
os.makedirs('C:/Users/vasyl/Downloads', exist_ok=True)
df_subset.to_csv("C:/Users/vasyl/Downloads/jobs2.csv")